<a href="https://colab.research.google.com/github/jdilger/useful_ee_scripts/blob/main/PlanetDownloaderForMosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
from pprint import pprint

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=bhfndSMKUxBzb-E8ji2AndEbt7eRC3lWB4a5nsOefP4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjfH5979K9cxZqm14akmQf4IFkzqaWwcBenMKXgNa_XOAv6sbLg_Sg

Successfully saved authorization token.


In [4]:
API = 'key here'
mosaics = !curl https://api.planet.com/basemaps/v1/mosaics?api_key=$API
mosaic_jsons = json.loads(mosaics[0])['mosaics']
print(mosaic_jsons[0])

In [8]:
# check date to see which mosaic you want
list_of_dates = list(map(lambda m: m['first_acquired'],mosaic_jsons))
print(list(map(lambda m: m['first_acquired'],mosaic_jsons)))
print(list_of_dates.index("2020-09-01T00:00:00.000Z"))


['2015-12-01T00:00:00.000Z', '2016-06-01T00:00:00.000Z', '2016-12-01T00:00:00.000Z', '2017-06-01T00:00:00.000Z', '2017-12-01T00:00:00.000Z', '2018-06-01T00:00:00.000Z', '2018-12-01T00:00:00.000Z', '2019-06-01T00:00:00.000Z', '2019-12-01T00:00:00.000Z', '2020-06-01T00:00:00.000Z', '2020-09-01T00:00:00.000Z', '2020-10-01T00:00:00.000Z', '2020-11-01T00:00:00.000Z', '2020-12-01T00:00:00.000Z', '2021-01-01T00:00:00.000Z', '2021-02-01T00:00:00.000Z', '2021-03-01T00:00:00.000Z', '2021-04-01T00:00:00.000Z', '2021-05-01T00:00:00.000Z', '2021-06-01T00:00:00.000Z', '2015-12-01T00:00:00.000Z', '2016-06-01T00:00:00.000Z', '2016-12-01T00:00:00.000Z', '2017-06-01T00:00:00.000Z', '2017-12-01T00:00:00.000Z', '2018-06-01T00:00:00.000Z', '2018-12-01T00:00:00.000Z', '2019-06-01T00:00:00.000Z', '2019-12-01T00:00:00.000Z', '2020-06-01T00:00:00.000Z', '2020-09-01T00:00:00.000Z', '2020-10-01T00:00:00.000Z', '2020-11-01T00:00:00.000Z', '2020-12-01T00:00:00.000Z', '2021-01-01T00:00:00.000Z', '2021-02-01T00:00:0

In [9]:
# selecting 2019-12-01T00:00:00.000Z @ position 8 
index_select = "2020-09-01T00:00:00.000Z"
moi = mosaic_jsons[list_of_dates.index(index_select)]
pprint(moi)
start_date = moi['first_acquired'][0:10]
end_date = moi['last_acquired'][0:10]
print(start_date, end_date)

{'_links': {'_self': 'https://api.planet.com/basemaps/v1/mosaics/c1ec34e1-4967-45a9-a2ba-6eba2d07cb8e?api_key=2e57fadb21854e2b8b2cf735ddad4cbe',
            'quads': 'https://api.planet.com/basemaps/v1/mosaics/c1ec34e1-4967-45a9-a2ba-6eba2d07cb8e/quads?api_key=2e57fadb21854e2b8b2cf735ddad4cbe&bbox={lx},{ly},{ux},{uy}',
            'tiles': 'https://tiles.planet.com/basemaps/v1/planet-tiles/planet_medres_normalized_analytic_2019-12_2020-05_mosaic/gmap/{z}/{x}/{y}.png?api_key=2e57fadb21854e2b8b2cf735ddad4cbe'},
 'bbox': [-179.999999974944,
          -30.145127179527,
          179.999999975056,
          30.145127179625],
 'coordinate_system': 'EPSG:3857',
 'datatype': 'uint16',
 'first_acquired': '2019-12-01T00:00:00.000Z',
 'grid': {'quad_size': 4096, 'resolution': 4.777314267823516},
 'id': 'c1ec34e1-4967-45a9-a2ba-6eba2d07cb8e',
 'interval': '1 mon',
 'item_types': ['PSScene4Band'],
 'last_acquired': '2020-06-01T00:00:00.000Z',
 'level': 15,
 'name': 'planet_medres_normalized_analyti

In [10]:
# bbox TerraBio pilot site
lx = -53.461751914154526
ux = -50.961740478449897
ly = -6.942135599911456
uy = -5.442128738488679

quads = moi['_links']['quads']
iname = moi['name']
print(quads,iname)

quad_formatted = quads.replace('{lx}',str(lx))\
                      .replace('{ly}',str(ly))\
                      .replace('{ux}',str(ux))\
                      .replace('{uy}',str(uy))
print(quad_formatted)

In [15]:
quad_fetch = requests.get(quad_formatted)
quad_fetch = quad_fetch.json()
pprint(quad_fetch['items'][0])

images = []
next = quad_formatted
while next:
  next_fetch = requests.get(next).json()
  images.extend(next_fetch['items'])
  # pprint(next_fetch['_links'])
  try:
    next = next_fetch['_links']['_next']
  except Exception as e:
    next = False

pprint('firest image :',images[0])
print('number of images :',len(images))

In [17]:
downloadurls = list(map(lambda quad: [quad['id'],quad['_links']['download']], images))
downloadurls.sort()
# downloadurls[0]
# # check the number of images to download
# print(len(downloadurls))
# print(downloadurls[143])

In [21]:
# manually created metadata for selected date
coldict = {

    'default' : {
        'date_start' : str(start_date),
        'date_end': str(end_date),
        'cloud' : 'gs://gee-upload/',
        'ee_dst':'projects/servir-amazonia/TerraBio/ImageCollections/Planet/'

    }
}
choice = 'default'
datestr = coldict[choice]['date_start']
datestrend = coldict[choice]['date_end']
# bandnames = 'blue,green,red,nir,alpha'

In [24]:
# range 0 to length of iamges to download
for i in range(0,len(downloadurls)):
  url = downloadurls[i][1]
  name = f"{downloadurls[i][0]}-{datestr}"

  # download the image and pipe it to a google coud bcket
  !wget {url} -O - | gsutil cp - {coldict['default']['cloud']}{datestr}-{datestrend}/{name}.tif
  # print(f"{coldict['default']['cloud']}{name}.tif",datestr, datestrend)

--2021-07-27 15:15:01--  https://api.planet.com/basemaps/v1/mosaics/c1ec34e1-4967-45a9-a2ba-6eba2d07cb8e/quads/719-984/full?api_key=2e57fadb21854e2b8b2cf735ddad4cbe
Resolving api.planet.com (api.planet.com)... 35.190.57.55
Connecting to api.planet.com (api.planet.com)|35.190.57.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/planet-mosaics-prod/planet_medres_normalized_analytic_2019-12_2020-05_mosaic/15/7ROB5S/719/984/data?Expires=1627485301&GoogleAccessId=mosaic-public-api-prod%40planet-datapipeline-prod.iam.gserviceaccount.com&Signature=UnpuSdvdzpTrAzAoCdJO%2FCZAh52eZyrxkjtT7TAjTBQ3ICtli0pVxO36FRprXgYuIHCK0dYZLNWehb6RlZzRJPBFCAGjS1twcfIC8aoydCHKApclEzC1qHSBGerpg1Pl2ivNELZz16mzZ4yUvv1TdrbT5f72t6EXOV5zpySD3p1Cz4q7WrDI%2BOkUa3A1uMPwKSb%2BL7J02LO0opjBdkR7dbp2URftEqN75R01kHFUP3tjeYEEPeFplQdlTS8i5EQDAhV%2F7U9SCQuGy34zzNt6BvSP2%2FxtNZb%2FN0HdVf04qW69YP9i%2FMiX8%2BYab127OSAhTcFBpaMWUKFmz66nI3nDoQ%3D%3D&response-content-disposit